In [1]:
from test_utils import *
from swayam import Agent

### Function Calling with Swayam Tools

Let's now start to link the functionality of an LLM Agent with a locally available function call.

#### Let's see what happens without a function call

In [2]:
from swayam import Swayam
Swayam.execute("What's the weather like in Boston?")

--------------------------------------------------------------------------------
Prompt: (Role: system)
--------------------------------------------------------------------------------
Prompt: (Role: system)
You are a helpful assistant. You are here to help me with my queries.
--------------------------------------------------------------------------------
Response:
Of course! I'm here to help you with any questions or queries you have. What do you need assistance with today?
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
--------------------------------------------------------------------------------
Prompt: (Role: user)
What's the weather like in Boston?
--------------------------------------------------------------------------------
Response:
I don't have real-time data access to provide current weather conditions. For the latest weather updates in 

ChatCompletionMessage(content="I don't have real-time data access to provide current weather conditions. For the latest weather updates in Boston, I recommend checking a reliable weather website or app.", refusal=None, role='assistant', function_call=None, tool_calls=None)

### Some Dummy Functions

Let's create some dummy functions

In [3]:
from enum import Enum
class UnitEnum(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit):
    import random
    i = random.randint(-10, 40)
    if unit.lower() == "celsius":
        return f"{i}°C"
    elif unit.lower() == "fahrenheit":
        return f"{i*9/5 + 32}°F"
    else:
        raise ValueError("Invalid unit")
    
def calculate(expression):
    return eval(expression)

def search_email(text):
    import re
    emails = re.findall(r'([\w\.-]+@[\w\.-]+)', text)
    if emails:
        return emails
    else:
        return None

## Tool Creation

As a part of an LLM workflow, we want to trigger the above local function based on response from the LLM (using function calling).

Before we can do that, we need to create a sort of bridge between a function from the view point of an LLM vs what it means in Python.

Here we see how we can create convert a function into a Tool.

### ToolBuilder

We use the tool builder to wrap the function created above.
1. Provide the function callable and describe it.
2. Add all arguments, while describing their types and purposes.

In [4]:
from swayam.llm.tool import ToolBuilder
builder = ToolBuilder(get_current_weather, desc="Get the current weather in a given location")
builder.add_field("location", type=str, desc="Location to get weather information for")
builder.add_field("unit", type=UnitEnum, desc="Unit of temperature", default=UnitEnum.CELSIUS)
get_current_weather_tool = builder.build()

builder = ToolBuilder(calculate, desc="Evaluates a mathematical expression")
builder.add_field("expression", type=str, desc="The expression to be evaluated.")
calc_tool = builder.build()

builder = ToolBuilder(search_email, desc="Searches email addresses in a given text")
builder.add_field("text", type=str, desc="The target text in which emails are to be searched.")
search_email_tool = builder.build()


### Tool Execution

The key thing to keep in mind is that all arguments are passed as keyword arguments irrespective of the underlying function.

In [5]:
get_current_weather_tool(location="dummy")

'40°C'

In [6]:
get_current_weather_tool(**{"location": "dummy"})

'2°C'

In [7]:
get_current_weather_tool(**{"location": "dummy", "unit":"fahrenheit"})

'73.4°F'

In [8]:
calc_tool(expression="2 + 5")

7

In [9]:
search_email_tool(text="This is a sample text with a@b.com address")

['a@b.com']

In [10]:
search_email_tool(text="This is a sample text with no email address")

### Tool Definition

One of the key benefits of creating a Tool is that it self describes itself, in a way compatible with an LLM.

In [11]:
print(get_current_weather_tool.definition)

{'type': 'function', 'function': {'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'properties': {'location': {'description': 'Location to get weather information for', 'title': 'Location', 'type': 'string'}, 'unit': {'description': 'Unit of temperature', 'enum': ['celsius', 'fahrenheit'], 'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}


In [12]:
print(calc_tool.definition)

{'type': 'function', 'function': {'name': 'calculate', 'description': 'Evaluates a mathematical expression', 'parameters': {'properties': {'expression': {'description': 'The expression to be evaluated.', 'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'type': 'object'}}}


In [13]:
print(search_email_tool.definition)

{'type': 'function', 'function': {'name': 'search_email', 'description': 'Searches email addresses in a given text', 'parameters': {'properties': {'text': {'description': 'The target text in which emails are to be searched.', 'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'type': 'object'}}}


Let's create the list of tool definitions to be shared with the LLM.

In [14]:
tools = [get_current_weather_tool, calc_tool, search_email_tool]

Let's create a tools dictionary so that once we know the name of the tool from LLM, we can call it with arguments returned from the LLM.

In [15]:
from swayam import Agent, Prompt

prompt = Prompt.as_system("You are a useful assistant. You are provided with a list of tools to help users with their queries.", tools=tools)
agent = Agent(system_prompt=prompt)
agent = Agent(display=True)

AttributeError: 'HtmlReporter' object has no attribute '_HtmlReporter__json'

In [ ]:
from swayam import Prompt
prompt = Prompt.as_user("What's the weather like in Boston?", tools=tools)

In [ ]:
function_json = agent.execute("What's the weather like in Boston?", functions=functions).function_call
tools[function_json.name](**json.loads(function_json.arguments))

In [ ]:
function_json = agent.execute("What is 7 *30 + 13?", functions=functions).function_call
tools[function_json.name](**json.loads(function_json.arguments))

In [ ]:
text = "This is a sample text with a@b.com and c@d.com addresses"
function_json = agent.execute(f"Find whether there is an email address in the following text marked with triple backticks \n ```{text}```?", functions=functions).function_call
tools[function_json.name](**json.loads(function_json.arguments))